In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [4]:
from nlpsig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [5]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [6]:
%run load_sbert-embeddings.py

In [7]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


## Baseline: Fine-tine BERT for classification

In [11]:
num_epochs = 10
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
split_indices = (df_rumours[df_rumours['set']=='train'].index,
                 df_rumours[df_rumours['set']=='dev'].index,
                 df_rumours[df_rumours['set']=='test'].index)

In [12]:
bert_classifier = fine_tune_transformer_average_seed(
    num_epochs=num_epochs,
    pretrained_model_name="bert-base-uncased",
    df=df_rumours,
    feature_name="text",
    label_column="label",
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_indices=split_indices,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/bert_classifier.csv",
    device=device,
    verbose=False
)

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.233061,0.676157,0.653401
2,No log,0.243724,0.669039,0.645784
3,No log,0.287661,0.626335,0.561570
4,No log,0.514238,0.633452,0.564223
5,No log,0.599408,0.626335,0.558711
6,No log,0.769594,0.647687,0.611276
7,No log,0.831784,0.615658,0.592491
8,0.083200,1.018997,0.637011,0.580671
9,0.083200,1.146123,0.640569,0.578272
10,0.083200,1.093049,0.644128,0.600353


  0%|          | 0/1049 [00:00<?, ?it/s]

...

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.231032,0.640569,0.608867
2,No log,0.270608,0.676157,0.652024
3,No log,0.285024,0.654804,0.641567
4,No log,0.553612,0.661922,0.589541
5,No log,0.739153,0.647687,0.551747
6,No log,0.865964,0.629893,0.582132
7,No log,1.038560,0.637011,0.563983
8,0.085300,0.958381,0.651246,0.597115
9,0.085300,1.073354,0.658363,0.595198
10,0.085300,1.110651,0.661922,0.595316


  0%|          | 0/1049 [00:00<?, ?it/s]

saving the results dataframe to CSV in rumours_output/bert_classifier.csv


In [13]:
bert_classifier

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.556721,0.553860,"[[0.589585172109444, 0.5181347150259067]]",0.554776,"[[0.571917808219178, 0.5376344086021505]]",0.554189,"[[0.6083788706739527, 0.5]]",1,focal,2,False
0,0.521449,0.520916,"[[0.504930966469428, 0.53690036900369]]",0.524413,"[[0.5505376344086022, 0.4982876712328767]]",0.524151,"[[0.4663023679417122, 0.582]]",12,focal,2,False
0,0.506196,0.499976,"[[0.5557461406518011, 0.44420600858369097]]",0.502144,"[[0.5251215559157212, 0.4791666666666667]]",0.502082,"[[0.5901639344262295, 0.414]]",123,focal,2,False


In [14]:
bert_classifier["f1"].mean()

0.52491722864066

In [15]:
bert_classifier["precision"].mean()

0.5271109575075326

In [16]:
bert_classifier["recall"].mean()

0.5268075288403157

In [17]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

array([[0.55008743, 0.49974703]])

In [18]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

array([[0.54919233, 0.50502958]])

In [19]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)

array([[0.55494839, 0.49866667]])

## Using Cross-Entropy loss

In [20]:
loss = "cross_entropy"
gamma = None

In [21]:
bert_classifier_ce = fine_tune_transformer_average_seed(
    num_epochs=num_epochs,
    pretrained_model_name="bert-base-uncased",
    df=df_rumours,
    feature_name="text",
    label_column="label",
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    split_indices=split_indices,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/bert_classifier.csv",
    device=device,
    verbose=False
)

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.606879,0.661922,0.628746
2,No log,0.823085,0.676157,0.569229
3,No log,0.844135,0.640569,0.595280
4,No log,0.990179,0.676157,0.629196
5,No log,1.180146,0.658363,0.614276
6,No log,1.398580,0.647687,0.618275
7,No log,1.698856,0.676157,0.609633
8,0.238900,1.711803,0.629893,0.588626
9,0.238900,1.863913,0.651246,0.592102
10,0.238900,1.913567,0.629893,0.577444


  0%|          | 0/1049 [00:00<?, ?it/s]

...

Map:   0%|          | 0/5568 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.623965,0.676157,0.658439
2,No log,0.779959,0.693950,0.644283
3,No log,0.854744,0.676157,0.622429
4,No log,1.343261,0.669039,0.575329
5,No log,1.346730,0.590747,0.541612
6,No log,1.653549,0.601423,0.549989
7,No log,1.976305,0.640569,0.578272
8,0.232100,2.338792,0.651246,0.538263
9,0.232100,2.372406,0.651246,0.554401
10,0.232100,2.372883,0.626335,0.542990


  0%|          | 0/1049 [00:00<?, ?it/s]

saving the results dataframe to CSV in rumours_output/bert_classifier.csv


In [24]:
bert_classifier_ce

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.476644,0.474027,"[[0.5111308993766697, 0.43692307692307686]]",0.474211,"[[0.5, 0.44842105263157894]]",0.474384,"[[0.5227686703096539, 0.426]]",1,cross_entropy,None,False
0,0.498570,0.457999,"[[0.6062874251497007, 0.3097112860892388]]",0.482497,"[[0.5146124523506989, 0.45038167938931295]]",0.486852,"[[0.7377049180327869, 0.236]]",12,cross_entropy,None,False
0,0.516683,0.509144,"[[0.4483133841131665, 0.5699745547073791]]",0.525801,"[[0.5567567567567567, 0.4948453608247423]]",0.523614,"[[0.37522768670309653, 0.672]]",123,cross_entropy,None,False


In [25]:
bert_classifier_ce["f1"].mean()

0.4803901043932053

In [26]:
bert_classifier_ce["precision"].mean()

0.49416955032551496

In [27]:
bert_classifier_ce["recall"].mean()

0.4949502125075895

In [28]:
np.stack(bert_classifier_ce["f1_scores"]).mean(axis=0)

array([[0.52191057, 0.43886964]])

In [29]:
np.stack(bert_classifier_ce["precision_scores"]).mean(axis=0)

array([[0.52378974, 0.46454936]])

In [30]:
np.stack(bert_classifier_ce["recall_scores"]).mean(axis=0)

array([[0.54523376, 0.44466667]])